# Course 1: Lidar
## Part 1: Introduction to Lidar and Point Clouds
#### By Jonathan L. Moran (jonathan.moran107@gmail.com)
From the Sensor Fusion Nanodegree programme offered at Udacity.

## Objectives

* Learn about LiDAR sensors, common LiDAR sensor types, and several LiDAR sensor defining characteristics;
* Learn about point clouds and their purpose in representing LiDAR sensor data;
* Understand several design considerations made when using LiDAR sensors on autonomous vehicles;
* Modify the [`SFND_Lidar_Obstacle_Detection`](https://github.com/udacity/SFND_Lidar_Obstacle_Detection/tree/master) codebase to perform several experiments on LiDAR sensor parameter selection and point cloud visualisation.

## 1. Introduction

### 1.1. The Lidar Sensor

[LiDAR](https://en.wikipedia.org/wiki/Lidar) (also "Lidar", an acronym for Light Detection and Ranging) is a method for estimating distances to objects using lightwaves. Most Lidar sensors conists of a [laser scanning](https://en.wikipedia.org/wiki/Laser_scanning) module and a [photodiode](https://en.wikipedia.org/wiki/Photodiode) module. In essence, these two modules work together to _transmit_ and _receive_ the pulsed laser light which is emitted by the sensor unit.

To estimate the distance to a nearby object, the Lidar sensor will first emit a laser beam in pulses or bursts. Assuming that nearby object(s) exist, these laser beams will then "bounce" or be reflected off of the surface of the object(s). The reflected laser beams are then returned to the sensor unit to be detected by the receiver (the photodiode). The time it took for the beam to be emitted and then returned to the sensor (i.e., the round-trip time) is calculated, from which the distance to the object can be estimated.

#### Types of Lidar Sensors

Lidar sensors can be categorised into different types based on their underlying technologies and operational principles.

When selecting a Lidar sensor for a specific application, one must weigh the various trade-offs that exist between Lidar sensor types against performance, cost and other characteristics to determine suitability.

Below is a list of trade-offs and considerations for the common types of Lidar sensors that exist today:


1. **Mechanical LiDAR**: a _rotating_ sensor unit that "spins" about an axis, typically generating a wide horizontal field-of-view (azimuth), e.g., $360^{\circ}$.
   * **Resolution**: High — mechanical LiDAR systems often offer high-resolution 3D scanning capabilities, making them suitable for applications requiring detailed point cloud data;
   * **Field-of-View**: Limited — due to the rotating scanning mechanism, mechanical LiDAR systems typically have a limited vertical field-of-view (FOV). This is due to the configuration of vertically-stacked laser diodes, where each is coupled with a receiver element. For example, the Velodyne VLP64 has $64$ vertically-stacked laser diodes. While this mechanical LiDAR unit has a comprehensive $360^{\circ}$ horizontal field-of-view, its vertical field-of-view is only $26.8^{\circ}$, which at farther distances may not be sufficient enough to resolve important details in the driving domain;
   * **Reliability**: Limited — mechanical LiDAR systems contain rotating parts which mechanically spin the laser-emitting diodes. Compared to non-rotating solid-state systems, these mechanical LiDAR systems are prone to additional wear-and-tear, potentially affecting the reliability and longevity of the sensor unit over time;
   * **Cost**: High — due to their complex design and moving parts, mechanical LiDAR units tend to be more expensive than the others.
2. **MEMS (Micro-Electrical-Mechanical) LiDAR**: a partially-rotating sensor unit consisting of many oscillating mirrors which scatter laser beams across a scene.
   * **Resolution**: Limited — both the resolution and the range may be limited with MEMS-based LiDAR systems. Since MEMS rely on an array of mirrors, there are several design considerations that affect resolution. Namely, the mirror size and the mirror oscillation affect the overall resolution of the sensor. Here, the speed, stability and sensitivity of the unit are crucial for maintaining sufficient resolution;
   * **Field-of-View**: Wide — due to the speed and accuracy in which the mirror elements can be tilted, MEMS LiDAR systems have the ability to scan large horizontal areas. However, this ability depends significantly on the optical design. This includes the arrangement of the mirrors and the lens design.  These design characteristics may be used to improve the laser density (via rapid beam deflection) and to ensure the optical path is optimised for satisfactory resolution (by reducing beam divergence and encouraging precise targeting - leading to smaller point sizes and higher resolution);
   * **Reliability**: Moderate — MEMS-based LiDAR systems have enhanced durability due to their solid-state nature (i.e., not requiring a rotating component), making them more resistant to shock and vibration over traditional mechanical LiDAR sensors. However, the micromirror design still presents concerns with respect to manufacturing defects or environmental factors. Also, MEMS is a newer technology and any conclusive long-term reliability data may not be available;
   * **Cost**: Moderate — can be more cost-effective when compared to other sensor types due to their simpler design and manufacturing processes.
3. **Frequency-Modulated Continuous Wave (FMCW) LiDAR**: a novel LiDAR system emitting "electronically steered" laser light without the use of moving parts.
   * **Resolution**: Moderate — FMCW LiDAR is able to offer improved resolution and efficiency due to its continuous-wave operation. By altering the "chirp rate" (rate of frequency change), the FMCW LiDAR can produce variable resolution at the cost of measurement time and range;
   * **Field-of-View**: Wide — FMCW LiDAR systems are able to electronically steer the antenna arrays to achieve a variable FOV. However, these mechanisms can add complexity and cost to the system;
   * **Reliability**: High — unlike traditional mechanical scanning LiDAR, FMCW LiDAR has good reliability due to the absence of moving parts. FMCW LiDAR systems may also be less sensitive to interference from sources such as sunlight, reflections, and other LiDAR units. In addition, FMCW LiDAR systems [have been shown](https://www.mobileye.com/blog/radar-lidar-next-generation-active-sensors/) to have better performance in common operating conditions, e.g., in adverse weather [3];
   * **Cost**: Moderate — can often be a moderate-cost option compared to high-resolution and mechanical LiDAR systems.
4. **Flash LiDAR**: entirely unlike scanning LiDAR systems, single-beam flash LiDAR systems emit an array of light, flooding the scene with light, which then illuminates every pixel in the sensor module simultaneously. Flash LiDAR is often compared to a camera with flash attachment.
   * **Resolution**: High — can achieve high resolution within its limited range [4];
   * **Field-of-View**: High — captures the entire scene in a single pulse;
   * **Reliability**: Moderate — Flash LiDAR is still a developing technology, which complicates the availability of long-term reliability studies;
   * **Cost** High — Flash LiDAR can come with a relatively high cost due to their advanced technology and complex design.


In this course we are going to primarily consider mechanical (rotating) LiDAR sensor units. Notably, the Velodyne HDL-64E [5], as shown in Figure 1 below:

<img src="figures/2024-04-14-Figure-1-Velodyne-HDL-64E.png" alt="Figure 1. Velodyne HDL-64E: Visualised." width="50%" height="50%">


$$\textrm{Figure 1. Velodyne HDL-64E: Visualised.}$$

#### Properties of Lidar Sensors

<img src="figures/2024-04-14-Figure-2-Velodyne-HDL-64E-Sensor-Specifications.png" alt="Figure 2. Velodyne HDL-64E: Specification Sheet." width="50%" height="50%">
$$\textrm{Figure 2. Velodyne HDL-64E: Specification Sheet.}$$

#### Quiz Question #1

**Question**: Given the above spec sheet shown in Figure 2, approximately how many points does an HDL-64E collect every second?

**Answer**: 2,880,000 points per second.

To calculate this answer, we multiply the number of lasers by the average update rate, i.e., $64 \times 10 Hz$. We must also account for the horizontal field-of-view, $360^{\circ}$, with respect to its angular resolution, $0.08^{\circ}$. In other words, the LiDAR sensor collects $64$ points for each of the $360^{\circ} / 0.08^{\circ} = 4500$ increments. This gives us a resulting $64 \times \left(4500 * 10 Hz\right) = 2,880,000$ points per second.

In [4]:
n_lasers = 64                 # Vertically-arranged
field_of_view = 360           # Degrees
angular_resolution = 0.08     # Degrees
update_rate_avg = 10          # Hertz

print("Total points collected every second: {:.2f}".format(
      n_lasers * field_of_view / angular_resolution * update_rate_avg
))

Total points collected every second: 2880000.00


### 1.2. Point Clouds


A [point cloud](https://en.wikipedia.org/wiki/Point_cloud) is a set of discrete data points in space. Here we consider point clouds to be the total set of data points captured by the LiDAR sensor for a given unit of time.

<img src="figures/2024-04-14-Figure-3-Point-Cloud-Rendering.png" alt="Figure 3. Rendering of a Point Cloud: A city block with parked cars and a passing van. Here, the _Intensity_ channel values colourise the LiDAR scanlines. The large spot near the centre containing very little data points corresponds to the location of the LiDAR sensor mounted atop the moving vehicle. The absence of data points in this circle indicates the challenges roof-mounted LiDAR units face in 'seeing' at near distances." height="50%" width="50%">

$\textrm{Figure 3. Rendering of a Point Cloud: A city block with parked cars and a passing van.}$

#### Point Cloud Basics

Point clouds usually consist of a set of 3D Cartesian coordinates $\left(x, y, z\right)$. In the case of LiDAR-generated data points, it is common to also have an [intensity value](https://geodetics.com/lidar-intensity-applications/#:~:text=LiDAR%20intensity%20is%20recorded%20as,object%20reflecting%20the%20laser%20beam.), $I$, recorded with each point. This Intensity value is analogous with the LiDAR signal strength returned to the sensor, and is often used to classify various properties of reflecting object surfaces.

In Figure 3 shown above, the _Intensity_ channel values colourise the LiDAR scanlines.

#### Point Cloud Tools

In order to store and represent the set of points contained in a point cloud, we use a common data file format known as the [Point Cloud Data](https://pointclouds.org/documentation/tutorials/pcd_file_format.html) format, which has file extension `.pcd`. The [Point Cloud Library](https://pointclouds.org/documentation/tutorials/index.html) (abbreviated "PCL") is used to read and modify `.pcd` files.

The Point Cloud Library contains many useful functions in C++ for visualising, segmenting, filtering, extracting and clustering the data points contained in a point cloud file. Common workflows involving PCL include object segmentation and ground plane separation.

### 1.3. Using Lidar on Autonomous Vehicles

LiDAR sensor systems can provide autonomous vehicles additional "eyes" with which to see the world around them. Leveraging LiDAR sensors, autonomous vehicles can quickly develop an "image" of their surroundings and use that image to avoid hitting pedestrians, animals, obstacles, and other vehicles.

While deep learning-based methods have advanced beyond standard 2D object detection, camera-only systems still struggle to accurately perceive 3D information. With LiDAR sensors, autonomous vehicles have the inherent ability to map out their environments in 3D and correctly piece together shapes for cars, people and other obstacles. The downsides of LiDAR over camera-based vision is the sensor's inability to distinguish between objects of similar sizes. For example, a LiDAR scan of a balloon floating in the centre of the road may appear analogous to an entirely different object of the same size, such as large rock [6]. While hitting a balloon may not pose any danger, colliding with a large rock can have severe consequences.

As with any other sensor modality, careful consideration must be made regarding the placement of the LiDAR sensor(s) onto an autonomous vehicle. As shown in Figure 3 above, mounting the LiDAR unit atop a vehicle (on its roof) may seem like the natural choice. However, it is readily apparent in the image that this choice results in a large "gap" in visibility in the area directly surrounding the sensor. Since there exists an absence of data points in this region, the vehicle will be unable to "see" anything in this vicinity. Therefore, many autonomous vehicle companies (Waymo, Cruise, etc.) will choose to "blanket" their vehicles in additional sensors to "fill in" the gaps directly around the vehicle. This also helps provide sensor redundancy and increases overall reliability, albeit at the cost of additional sensor units and fusion algorithm complexity.

Also factored into the LiDAR sensor equation is the granularity or detail with which the sensor can "see" into its environment. Depending on the vertical field-of-view (FOV) and laser diode array, a LiDAR sensor may have "gaps" in its vision that become increasingly blinding as the distance increases. For example, the 16-diode Velodyne VLP-16 has an "angle of separation" of about ~2 degrees, that, when projected outward, will create a 'hidden space' in which no LiDAR points are recoverable (similar to what is shown in the negative space surrounding the sensor from Figure 3). On the Velodyne 16-channel VLP-16, with approximately 2 degrees of separation (i.e., the vertical angular resolution), a large enough "gap" between the diodes will be created such that at, say, 60 metres ahead, a pedestrian or other obstacle may be unresolvable.

<img src="figures/2024-04-14-Figure-4-Waymo-Sensor-Configuration.png" alt="Figure 4. Sensor configuration of the Waymo autonomous vehicle." height="50%" width="50%">

$$\textrm{Figure 4. Sensor configuration of the Waymo autonomous vehicle (credit: Jonathan L. Moran).}$$

Shown in Figure 4 is an example sensor configuration on a passenger vehicle. In this depiction of Waymo's autonomous vehicle we see there are five roof-mounted RGB cameras, one roof-mounted LiDAR sensor and four additional LiDAR sensors mounted on the body of the vehicle. More on this sensor particular configuration is available in my Sensor Fusion course notebook [here]( (https://github.com/jonathanloganmoran/ND0013-Self-Driving-Car-Engineer/blob/main/2-Sensor-Fusion/Exercises/2022-10-20-Course-2-Sensor-Fusion-Exercises-Part-1.ipynb).

#### Quiz Question #2

**Question**: While scanning a Velodyne HDL-64E, a laser signal from the top layer takes $66.7 ns$ to be emitted and received again. The laser is emitted at a $-24.8^{\circ}$ incline from the $x$-axis, which then travells horizontally along the $x$-axis. Knowing that the speed of light is $299792458 \ m/s$, what would be the Cartesian coordinates of this laser point, $\left(x_{1}, y_{1}, z_{1}\right)$, given in metres?


<img src="figures/2024-04-14-Figure-5-Coordinate-System-LiDAR-Vehicle.png" alt="Figure 5. Coordinate System for the LiDAR-Vehicle frame.">

$$\textrm{Figure 5. Coordinate System for the LiDAR-Vehicle frame.}$$

**Answer**: $\left(9.08, 0.0, -4.19\right)$.

To calculate this answer, we must first define the coordinate system with respect to the LiDAR-Vehicle reference frame. As shown in Figure 5, we have the $x$-axis parallel with the ground plane, and the $z$-axis perpendicular to the $x$-axis. We will assume that the reference point (i.e., the starting coordinate) exists at: $$\left(x_{0}, y_{0}, z_{0}\right) = \left(0, 0, 0\right).$$ This is the point at which the laser is emitted from the sensor positioned atop the vehicle.

In the problem statement we are told that the laser beam is emitted at an angle: $$\theta_{0} = -24.8^{\circ},$$ which is defined to be an incline from the $x$-axis. We are also told that the beam travells horizontally along the $x$-axis. Using this, we can assume that the displacement of the beam along the $y$-axis will be zero throughout its journey. Therefore, we have: $$y_{1} = 0.0.$$

Now to find the remaining two coordinate values $x_{1}$ and $z_{1}$ of the reflecting object, we use trigonometry to decompose the one-way distance travelled by the laser into its $x$- and $z$ components. Knowing that our angle of departure, $\theta_{0} = -24.8^{\circ}$, is at an incline with the $x$-axis, we use the [Pythagorean theorem](https://en.wikipedia.org/wiki/Pythagorean_theorem) to form a [right triangle](https://en.wikipedia.org/wiki/Right_triangle) such that the hypotenuse represents the one-way distance travelled by the laser. By applying Pythagorean's theorem, we can relate the lengths of the $x$- and $z$ components of the beam (the "legs" of the right triangle) to the hypotenuse — our one-way distance travelled by the laser.

First, let's calculate the one-way distance travelled by the laser. The time the laser beam "took" to reach the object in-question can be considered to be simply one-half the total round-trip travel time, which is given as $66.7 ns$. Since we prefer to compute this in [SI units](https://en.wikipedia.org/wiki/SI_base_unit), we will represent nanoseconds in its equivalent form: $\mathrm{RTT} = 66.7 \times 10^{-9}$ seconds. Now, to obtain the one-way time, we take half the round-trip time. Therefore, we have: $$t = \frac{1}{2} * \left(6.67 \times 10^{-9}\right) = 3.335 \times 10^{-9} s.$$

Let's now take our one-way travel time $t$ and multiply it by the given [speed of light](https://en.wikipedia.org/wiki/Speed_of_light), $c = 299792458 \ m/s$, to get the estimated distance travelled $d$. Using $d = c \times t$, we obtain: $$d = \left(299792458 \ m/s\right) \times \left(3.335 \times 10^{-9} ns\right), \\ d \approx 9.98 m.$$

Forming the right-triangle we discussed early such that the hypotenuse represents the one-way distance travelled $d$ we computed above, we represent the angle of departure $\theta_{0} = -24.8$ with the corrected angle $\theta_{c} = 90^{\circ} - \theta_{0}$, since the angle given to us is negative.

All that is left for us to do is to find the individual components of the laser beam along the triangle: $$x_{1} = d\times sin\left(\theta_{c}\right), \\ z_{1} = d\times cos\left(\theta_{c}\right).$$

This gives us the following:
$$x_{1} = 9.98 m \ \times sin\left(65.2^{\circ}\right) \approx 9.06 m, \\ z_{1} = 9.98 m \times cos\left(65.2^{\circ}\right) \approx -4.19 m.$$


Note that the negative sign for the z-coordinate indicates that the destination point is below the origin of the $x$-axis.

Therefore, we obtain the Cartesian coordinates (in metres) of the laser point reflected off of the object to be approximately $$\therefore \left(x_{1}, y_{1}, z_{1}\right) = (9.06, 0.0, -4.19).$$

## 2. Programming Task

### 2.1. Lidar Sensor Simulator

In this module we will use a basic simulation engine built in C++. This programme will allow us to simulate the optical effects of a LiDAR sensor using [ray tracing](https://en.wikipedia.org/wiki/Ray_tracing_(graphics)). We will also be able to modify some of the design parameters of the sensor itself — resolution, minimum scanning distance — to better make use of the sensor for our intended use case: a simple highway driving scenario. To construct this scene, we will use basic graphical elements to form vehicle objects, traffic lanes, and traffic lane lines. This basic demonstration will give us an idea of how LiDAR sensors work, i.e., how they scan their surroundings and what their data products look like.  

#### The project codebase

The codebase we will be working with is hosted on GitHub at [this repository](https://github.com/udacity/SFND_Lidar_Obstacle_Detection) [7]. This repository relies on the C++ [Point Cloud Library](https://pointclouds.org/) [8] to render the scene and simulate the LiDAR sensor.

The file structure and some comments on the individual files we will be working with is shown below in Figure 6:

```
-- SFND_Lidar_Obstacle_Detection
    |-- CMakeLists.txt
    |-- README.md
    |-- build
    |   `-- environment
    `-- src
        |-- environment.cpp                 // Entry point; contains the `main` function.
        |-- processPointClouds.cpp          // Processes and visualises the PCL data.
        |-- processPointClouds.h
        |-- render
        |   |-- box.h                       // Struct definitions for cars (cuboids).
        |   |-- render.cpp                  // Renders and modifies scene objects.
        |   `-- render.h
        `-- sensors
            |-- data
            |   `-- pcd
            |       |-- data_1
            |       |   |-- *.pcd
            |       |-- data_2
            |       |   |-- *.pcd
            |       `-- simpleHighway.pcd
            `-- lidar.h                     // Simulates sensor functionality (e.g., ray casting).
```


$$\textrm{Figure 6. File structure of the } \texttt{SFND_Lidar_Obstacle_Detection } \textrm{project}.$$

Most of our work will be done in the [`environment.cpp`](https://github.com/udacity/SFND_Lidar_Obstacle_Detection/blob/master/src/environment.cpp) file. In this file there are several functions used to create and modify our 3D world — the simple highway scene — and control the LiDAR sensor we are simulating.

#### E1.1.0-1: Creating a "simple highway" scene

To test out our simulated LiDAR sensor object, we will first create a very basic highway driving scene. This scene consists of several vehicles and highway lanes. To render this scene, we will use the [`simpleHighway()`](https://github.com/udacity/SFND_Lidar_Obstacle_Detection/blob/master/src/environment.cpp#L38) function. This function creates a [`pcl::visualization::PCLVisualizer::Ptr`](http://pointclouds.org/documentation/classpcl_1_1visualization_1_1_p_c_l_visualizer.html) instance, which is simply the 3D scene viewer or "canvas" onto which we will render the "highway" scene objects. 

Calling the [`initHighway()`](https://github.com/udacity/SFND_Lidar_Obstacle_Detection/blob/master/src/environment.cpp#L11) function returns us a vector of `Car` objects. By default, this vector should now contain four `Car` objects: one "ego" vehicle, and three neighbouring vehicles.


Our first task will be to modify the `simpleHighway()` function to create a LiDAR sensor instance. To do so, we call the `Lidar()` constructor, parameterising it with two arguments: the `cars` vector (returned by the `initHighway()` function), and a `groundSlope` decimal point value. This ground slope value is used to define the angle at which the Lidar sensor will project outward a "ray" of laser light. This laser light beam is projected at an angle `groundSlope` radians from the $x$-$y$ plane. For example, an angle of $\texttt{groundSlope} = \pi/2$ would correspond to a ray projected straight up the $x$-$y$ plane (at $90^{\circ}$ from the origin).

Completing this, we have the following:

```cpp
// In `/src/environment.cpp`:

#include "sensors/lidar.h"      // Sensor instantiation

void simpleHighway(
    pcl::visualization::PCLVisualizer::Ptr& viewer
) {
  // ..
  /** E1.1.0: Create 3D highway scene. **/
  // RENDER OPTIONS
  bool renderScene = true;
  std::vector<Car> cars = initHighway(
      renderScene, 
      viewer
  );
  /** E1.1.1: Create LiDAR sensor object. **/
  // SENSOR SPECIFICATIONS
  float groundSlope = 0.0;                        // Radians; angle of departure between $x$-$y$ plane.
  // TODO:: Create lidar sensor
  Lidar *lidar = new Lidar(
      cars,
      groundSlope
  );
}
```

<img src="figures/2024-04-14-Figure-7-Simple-Highway-Simulation-Rendering.png" alt="Figure 7. Simple Highway Simulation: Rendering of the highway scene.">

$$\textrm{Figure 7. Simple Highway Simulation: Rendering of the highway scene.}$$

#### E1.1.2-3: Using the simulated Lidar sensor

In this exercise we make use of the `Lidar` object to perform several key functions needed to visualise both the laser rays emitted from the sensor, and the point cloud generated by the sensor. 

To begin this exercise, we perform a "scan" of the environment. A "scan" refers to the process by which the sensor emits laser beams, collects the returned laser beams, and calculates the distance(s) to the object(s) which the beams were reflected off of. To perform this operation, we use the class function `lidar->scan()` which returns a PointCloud (`pcl::PointCloud<pcl::PointXYZ>::Ptr`) object pointer.

Once we obtain our first scan, we'll want to visualise the laser beams ("rays") which were emitted from the sensor onto the detected points in the scene. To do this, we call the `renderRays()` function.

Lastly, to display the points detected in the point cloud (i.e, the point cloud measurements), we call the `renderPointCloud()` function.

```cpp
// In `/src/environment.cpp`:

void simpleHighway(
    pcl::visualization::PCLVisualizer::Ptr& viewer
) {
  // ...
  /** E1.1.2: Project the LiDAR rays onto the scene. **/
  // Generate a new Lidar sensor scan
  pcl::PointCloud<pcl::PointXYZ>::Ptr pointCloud = lidar->scan();
  // Project the LiDAR sensor rays onto the scene
  renderRays(
      viewer,
      lidar->position,
      pointCloud
  );
  /** E.1.1.3: Display the detected points. **/
  renderPointCloud(
      viewer,
      pointCloud,
      "pointCloud"
  );
}
```

<img src="figures/2024-04-14-Figure-8-Simple-Highway-Simulation-LiDAR-Ray-Projection-and-Point-Cloud-Rendering.png" alt="Figure 8. Simple Highway Simulation: Projected rays (top) and point cloud data (bottom) obtained from the simulated Lidar sensor.">

$$\textrm{Figure 8. Simple Highway Simulation: Projected rays (top) and point cloud data (bottom) obtained from the simulated Lidar sensor.}$$

#### E1.1.4-7: Modifying the simulated Lidar sensor parameters

There are several LiDAR sensor parameters that we can modify in the [`/src/sensors/lidar.h`]() file. In this set of exercises, we will "increase" the _scan resolution_ of our LiDAR sensor. To do so, we will change the values set for the `numLayers` and `horizontalAngelInc` variables. We will also simulate sensor measurement _uncertainty_ by adding sensor "[noise](https://en.wikipedia.org/wiki/Noise_(signal_processing))" using the `sderr` parameter.

Adjusting these variables can help tailor the LiDAR sensor's scanning characteristics to meet the specific requirements of our sensor unit or to better model the environment in which the sensor will operate.

##### E1.1.4: `numLayers`

In our project we assume the `numLayers` variable represents the number of vertical layers, or beams, emitted by the simulated LiDAR sensor. By increasing the `numLayers` parameter, we effectively increase the vertical resolution of the sensor by increasing the number of vertical rays emitted. Each "layer" here corresponds to a beam being emitted from the sensor module. Each beam generates a circular "scan line" onto which points can be detected. Therefore, we can add additional scan lines by increasing the `numLayers` parameter, which could improve the sensor's scanning resolution.

```cpp
// In `src/sensors/lidar.h`:

Lidar(std::vector<Car> setCars, double setGroundSlope)
		: cloud(new pcl::PointCloud<pcl::PointXYZ>()), position(0,0,2.6)
	{
		/*** E1.1.4-8: Modify the LiDAR sensor parameters. ***/
        // ...
        /** E1.1.4: Increase the vertical scanning resolution. **/
        // TODO:: increase number of layers to 8 to get higher resoultion pcd
        // Modifying the number of vertical beams to increase resolution
        // int numLayers = 3;  
        int numLayers = 8;                      	// Number of vertically-stacked diodes
        // ...
```

## 3. Closing Remarks

#### Alternatives

#### Extensions of task

## 4. Future Work

## Credits

This assignment was prepared by Aaron Brown and Michael Maile of Mercedes-Benz Research & Development of North America (MBRDNA), 2021 (link [here](https://learn.udacity.com/nanodegrees/nd313/)).


References
* [1] The Next Generation of Active Sensors for Autonomous Driving. Mobileye. 2021. https://www.mobileye.com/blog/radar-lidar-next-generation-active-sensors/.
* [2] Things you need to know about LiDAR: solid-state and hybrid solid-state, what's the difference? Hesai Group. 2023. https://www.hesaitech.com/things-you-need-to-know-about-lidar-solid-state-and-hybrid-solid-state-whats-the-difference/.
* [3] At the heart of sensors — MEMS technology for LiDAR. Mathias Müller. Blickfield Blog. 2020. https://www.blickfeld.com/blog/mems-technology-for-lidar/.
* [4] How Multi-Beam Flash Lidar Works. Angus Pacala. Ouster Inc. 2018. https://ouster.com/insights/blog/how-multi-beam-flash-lidar-works.
* [5] High Definition Lidar: Velodyne HDL-64E. Hypertech Co. 2014. https://hypertech.co.il/wp-content/uploads/2015/12/HDL-64E-Data-Sheet.pdf.
* [6] What Lidar Is and Why It's Important For Autonomous Vehicles. Chris Teague. Autoweek. 2021. https://www.autoweek.com/news/a36190274/what-lidar-is/.
* [7] `SFND_Lidar_Obstacle_Detection`. @udacity. GitHub. 2022. https://github.com/udacity/SFND_Lidar_Obstacle_Detection/tree/master.
* [8] Point Cloud Library. Radu Bogdan Rusu, Steven Cousins. 2011. https://pointclouds.org/about/.